In [ ]:
#トラカンの最近傍の値と距離を計算し、マッチさせていく(検証版2)

In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import psycopg2
import datetime

In [17]:
dbname = "kddi_18_param_custom2"
npyname = "trkn_distance"
datedir = "230120"
sttime = "2022-04-01 18:00:00"

In [150]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [ ]:
#特定日付のトラカンデータを全取得 > 緯度経度
cur =connection.cursor()
sql = "SELECT trakanmap.gridcode, 断面交通量,pointdata.latitude,pointdata.longitude , trakanmap.geom FROM trakan2204 "
sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
sql += "WHERE 時刻 = '"+sttime+"' AND trakanmap.gridcode != '0' "
sql += "ORDER BY trakanmap.gridcode"
cur.execute(sql)
trknDf = pd.DataFrame(cur.fetchall())
cur.close()

In [81]:
#特定日付のKDDIメッシュ情報を全取得
cur =connection.cursor()
sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_18_param  "
sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
sql += "WHERE times = '"+sttime+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
cur.execute(sql)
kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
cur.close()

In [152]:
curwrite =connection.cursor()

#KDDIをループ
for kddiIndex, kddiRow in kddiDf.iterrows():
    kddilat = str(kddiRow[2])
    kddilng = str(kddiRow[3])
    #print(str(kddiRow[0]))
    #直近のトラカンの位置と、その角度を求める
    cur =connection.cursor()
    sql = "SELECT trakanmap.gridcode, pointdata.latitude,pointdata.longitude,断面交通量,"
    sql += "ST_Distance( pointdata.geom, ST_GeographyFromText('SRID=4612;POINT("+kddilng+" "+kddilat+")')) AS distance , "
    sql += "st_azimuth(pointdata.geom, ST_GeographyFromText('SRID=4612;POINT("+kddilng+" "+kddilat+")')) /(2*pi())*360 AS angle "
    sql += "FROM trakan2204 "
    sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
    sql += "INNER JOIN pointdata on pointdata.grid_code = trakanmap.gridcode "
    sql += "WHERE ST_DWithin(pointdata.geom, ST_GeographyFromText('SRID=4612;POINT("+kddilng+" "+kddilat+")'), 1000000.0) "
    sql += "and 時刻 = '"+sttime+"' AND trakanmap.gridcode != '0' "
    sql += "ORDER BY distance limit 1"
    cur.execute(sql)
    nearDf = pd.DataFrame(cur.fetchall())
    cur.close()
    finds = 0
    for nearIndex, nearRow in nearDf.iterrows():
        nearGrd = str(nearRow[0])
        kddiParam = kddiDf.query('mesh == '+nearGrd)
        if len(kddiParam) == 0:
            print(nearGrd + " is kddi none!")
            continue
        finds = 1
        print(nearRow)
        nearTbran = kddiParam['tbran'].iloc[-1] #kddi
        nearlat = str(nearRow[1])
        nearlng = str(nearRow[2])
        nearDanmen = nearRow[3] #trkn
        nearDst = nearRow[4]
        nearAngle = nearRow[5]
        #逆向きに置換
        if (nearAngle + 180) >= 360:
            nearAngle = nearAngle - 180
        else:
            nearAngle = nearAngle + 180
        nearAngle = str(nearAngle)
        
        #直近のトラカン位置から半径5kmのトラカン位置と角度を求める
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode, pointdata.latitude,pointdata.longitude,断面交通量, "
        sql += "ST_Distance( pointdata.geom, ST_GeographyFromText('SRID=4612;POINT("+nearlng+" "+nearlat+")')) AS distance , "
        sql += "abs(st_azimuth(pointdata.geom, ST_GeographyFromText('SRID=4612;POINT("+nearlng+" "+nearlat+")')) /(2*pi())*360 - " + nearAngle  + ") AS angle "
        sql += "FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata on pointdata.grid_code = trakanmap.gridcode "
        sql += "WHERE ST_DWithin(pointdata.geom, ST_GeographyFromText('SRID=4612;POINT("+nearlng+" "+nearlat+")'), 5000.0) "
        sql += "and trakanmap.gridcode <> '" +nearGrd+ "' AND 時刻 = '"+sttime+"' AND trakanmap.gridcode != '0' "
        sql += "ORDER BY angle limit 1"
        cur.execute(sql)
        areatrknDf = pd.DataFrame(cur.fetchall())
        cur.close()
        
        find = 0
        for areatrknIndex, areatrknRow in areatrknDf.iterrows():
            if len(kddiParam) == 0:
                print("kddiPtParam is zero")
                break
            kddiParam = kddiDf.query('mesh == '+areatrknRow[0])
            nextTbran = kddiParam['tbran'].iloc[-1] #kddi
            nextDanmen = areatrknRow[3]  #trkn
            nextDst = areatrknRow[4] 
            find = 1
        
        if find == 0:
            print("trkn data none!")
        #pprint("baseTbran = " + str(kddiRow[1]))
        #pprint("nearTbran = " + str(nearTbran))
        #pprint("nextTbran = " + str(nextTbran))
        #print("nearDanmen = " + str(nearDanmen))
        #print("nextDanmen = " + str(nextDanmen))
        #print("nearDst = " + str(nearDst))
        #print("nextDst = " + str(nextDst))

        #断面が大きいほうから小さいほうを引いて差を求める
        diffDanmen = 0
        baseDanmen = 0
        if nearDanmen > nextDanmen:
            diffDanmen = nearDanmen - nextDanmen
            baseDanmen = nextDanmen
        else:
            baseDanmen = nearDanmen
            diffDanmen = nextDanmen - nearDanmen

        #比率から加算値を求める
        diffDist = round(nearDst / nextDst * diffDanmen)
        #元値を無視する？
        #print("baseDanmen = " + str(baseDanmen))
        #print("diffDist = " + str(diffDist))
        #print("setDist = " + str(baseDanmen + diffDist))
        sql = "INSERT INTO kddi_18_distparam (times, mesh, tbran) VALUES ('"+sttime+"',"  + str(kddiRow[0]) + ","  + str(baseDanmen + diffDist) + ");"
        curwrite.execute(sql)
        connection.commit()
curwrite.close()



TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
print("ok")

In [147]:
#見つかったトラカンをループしながら、最近傍のデータとの緯度、経度を取得
#そのエリア内に含まれるKDDIメッシュ情報を取得
for trknIndex, trknRow in trknDf.iterrows():
    cur =connection.cursor()
    sql = "SELECT trknmesh2,meter,pointdata.latitude,pointdata.longitude "
    sql += "FROM trkn_distance "
    sql += "INNER JOIN pointdata ON trknmesh2 = cast(pointdata.grid_code AS BIGINT) "
    sql += "WHERE trknmesh1 = '"+trknRow[0]+"' ORDER BY meter LIMIT 1"
    cur.execute(sql)
    pathDf = pd.DataFrame(cur.fetchall())
    cur.close()
    
    cur =connection.cursor()
    sql = "SELECT trknmesh2,meter,pointdata.latitude,pointdata.longitude "
    sql += "FROM trkn_distance "
    sql += "INNER JOIN pointdata ON trknmesh2 = cast(pointdata.grid_code AS BIGINT) "
    sql += "WHERE trknmesh1 = '"+trknRow[0]+"' ORDER BY meter LIMIT 1"
    cur.execute(sql)
    pathDf = pd.DataFrame(cur.fetchall())
    cur.close()
    
#起終点のメッシュの重みはトラカン値そのままを利用
#縦と横のメッシュ数を計算
#該当のKDDIメッシュから、トラカン起終点の距離を取得
#起点までの距離と終点までの距離から比率を求める
#自身の重みを起終点から比率配分し平均化する。



NameError: name 'trknDf' is not defined

In [ ]:
connection.close()